In [1]:
#BPR
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial
from tqdm import tqdm
%matplotlib inline

In [2]:
DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
DATA_FOR_BPR = "data_for_BPR.csv"
SAVE_DIR = "../gamedata/train_result/"

In [3]:
# #data preprocessing
# data = pd.read_csv(DATA_DIR+DATA_FILE)
# gameid = data['gameid'].drop_duplicates()
# gameid_list = []
# gameid_list = list(gameid)
# gameid2newid = {old:new+1 for new, old in enumerate(gameid_list)}
# data['new_game_id'] = data['gameid'].map(gameid2newid)

# events = data['eventname'].drop_duplicates()
# events_list = list(events)
# events2id = {old:new+1 for new,old in enumerate(events_list)}
# data['new_event_id'] = data['eventname'].map(events2id)

In [ ]:
#data.to_csv(DATA_DIR+'data_for_BPR.csv',index=False)

In [ ]:
#data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [ ]:
#data[data['new_game_id']==63]

In [4]:
def load_data(data_path):
    game_events = defaultdict(set)
    game_count = -1
    event_count = -1
    with open(data_path,'r') as f:
        header_line = next(f)
        for line in f.readlines():
            _, _, _, _, _, _, gameid, eventid = line.split(",")
            gameid = int(gameid)
            eventid = int(eventid)
            #print(u,i)
            game_events[gameid].add(eventid)
            game_count = max(gameid, game_count)
            event_count = max(eventid, event_count)
    print ("game_count:", game_count)
    print ("event_count:", event_count)
    #return max_u_id, max_i_id, user_ratings
    return game_count,event_count,game_events
    

#data_path = os.path.join('D:\\tmp\\ml-100k', 'u.data')
#user_count, item_count, user_ratings = load_data(data_path)
game_count,event_count,game_events_dict = load_data(DATA_DIR+DATA_FOR_BPR)

game_count: 1108
event_count: 11189


In [5]:
#
def generate_test(game_events_dict):
    user_test = dict()
    for u, i_list in game_events_dict.items():
        user_test[u] = random.sample(game_events_dict[u], 1)[0]
    return user_test

game_test_set = generate_test(game_events_dict)

In [6]:
#generate dataset for training
def generate_train_batch(game_events_dict, game_test_set, event_count, batch_size):
    t = []
    
    #print('generate batch..')
    for b in range(batch_size):
        break_flag1 = 0
        break_flag2 = 0
        u = random.sample(game_events_dict.keys(), 1)[0]
        #print('u:',u)
        i = random.sample(game_events_dict[u], 1)[0]
        while i == game_test_set[u]:
            break_flag1 += 1
            i = random.sample(game_events_dict[u], 1)[0]
            if break_flag1 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('i:',i)
        j = random.randint(1, event_count)
        while j in game_events_dict[u]:
            break_flag2 += 1
            j = random.randint(1, event_count)
            if break_flag2 == 20:
                #game_events_dict.pop('u',None)
                #game_test_set.pop('u',None)
                break
            #print('j:',j)
        t.append([u, i, j])
        #print(u,i,j)
    #print('generate batch done!')
    return numpy.asarray(t)

#train_set = generate_train_batch(game_events_dict,game_test_set,event_count)

In [10]:
def generate_test_batch(game_events_dict, game_test_set, event_count):
    for u in game_events_dict.keys():
        t = []
        i = game_test_set[u]
        #for j in range(1, event_count+1):
        for j in range(1, 501):
            if not (j in game_events_dict[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

In [11]:
def bpr_mf(game_count, event_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [game_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [event_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keepdims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user
    # 
    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

In [12]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(game_count, event_count, 20)
#     print(u, i, j, mf_auc, bprloss, train_op)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('start epoch')
    for epoch in range(1, 101):
        print ("epoch: ", epoch)
        _batch_bprloss = 0
        for k in tqdm(range(1, 5000)): # uniform samples from training set
            uij = generate_train_batch(game_events_dict, game_test_set,event_count,batch_size=128)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        games_count = 0
        _auc_sum = 0.0
        test_bprloss = 0.0
        # each batch will return only one user's auc
        for t_uij in generate_test_batch(game_events_dict, game_test_set, event_count):

            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                    feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]}
                                )
            games_count += 1
            _auc_sum += _auc
            test_bprloss += _test_bprloss
#            print ("test_loss: ", test_bprloss/games_count)
        print ("test_loss: ", test_bprloss/games_count, "test_auc: ", _auc_sum/games_count)
        print ("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    saver.save(session, SAVE_DIR)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)
print('all done')


  0%|          | 5/4999 [00:00<01:40, 49.62it/s]

start epoch
epoch:  1



100%|██████████| 4999/4999 [00:25<00:00, 196.48it/s]


bpr_loss:  0.7007177647744972
_train_op


  0%|          | 23/4999 [00:00<00:22, 224.68it/s]

...

bpr_loss:  0.3984532572431883
_train_op


  0%|          | 22/4999 [00:00<00:22, 216.89it/s]

test_loss:  0.6443427161780936 test_auc:  0.7872218031558924

epoch:  100


100%|██████████| 4999/4999 [00:23<00:00, 208.89it/s]


bpr_loss:  0.39737753573358714
_train_op
test_loss:  0.6439241473747935 test_auc:  0.7870936349785755

Variable:  user_emb_w:0
Shape:  (1109, 20)
[[ 0.13717215 -0.00894479  0.1064348  ...  0.10948678 -0.08258834
   0.04392402]
 [ 0.12536527  0.21623893 -0.44520482 ...  0.03649102  0.42791945
  -0.06447671]
 [ 0.16242956  0.0752534  -0.30374938 ... -0.16885716  0.40871626
   0.5473106 ]
 ...
 [ 0.09247427  0.21443774 -0.41464174 ... -0.07634244  0.21161343
   0.30089897]
 [ 0.20118609  0.23916088 -0.32743484 ...  0.06709398  0.25478318
   0.20179431]
 [ 0.11073586  0.02372987  0.05191761 ...  0.18937714  0.08402411
  -0.01127102]]
Variable:  item_emb_w:0
Shape:  (11190, 20)
[[ 0.0131405   0.02842092 -0.09328394 ...  0.01052712 -0.05503642
   0.02240199]
 [ 0.50951964  0.5788157  -0.7668364  ...  0.1881455   0.8428015
   0.42268595]
 [ 0.48794967  0.68124795 -0.90599966 ...  0.07878713  0.6939764
   0.680553  ]
 ...
 [ 0.12171526 -0.23565903  0.13592367 ...  0.22325248 -0.11451741
   0.2

In [13]:
print(len(t_uij))
print(len(uij))

495
128


In [29]:
_test_bprloss

1.1672156

In [ ]:
saver.save(session, SAVE_DIR)

In [71]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][200], 0)
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)
result_1 = session1.run(u1_all)
print (result_1)

[[-0.02186481  2.421619    2.3348546  ... -0.14870542  0.01071435
   0.04085217]]


In [72]:
data=pd.read_csv(DATA_DIR+'data_for_BPR.csv')

In [73]:
print("new recommedation ：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-40]] = 0
for index in range(len(p)):
    if p[index] != 0:
        select = data[data['new_event_id']==index].drop_duplicates('eventname')
  
        print (index, p[index],select['eventname'].values)

new recommedation ：
1 2.421619 ['transaction']
2 2.3348546 ['gameEnded']
3 2.2810893 ['missionFailed']
4 2.350466 ['newPlayer']
5 2.408208 ['levelUp']
8 1.4797173 ['social']
9 2.3437192 ['clientDevice']
10 2.3339155 ['missionCompleted']
11 2.0723672 ['missionAbandoned']
12 2.2987897 ['gameStarted']
13 1.5776081 ['inviteSent']
14 1.2136604 ['inviteReceived']
15 2.3330386 ['missionStarted']
16 1.641531 ['achievement']
20 1.3275536 ['itemActioned']
21 2.0999913 ['options']
22 1.7021737 ['shopEntered']
50 1.4089695 ['giftSent']
82 2.0402346 ['notificationServices']
139 1.5713735 ['socialConnect']
143 1.167326 ['giftReceived']
148 2.0753944 ['appsFlyerAttribution']
150 1.4034182 ['tutorialStep']
158 2.0732374 ['engageResponse']
170 1.4192733 ['itemCollected']
209 1.3456204 ['matchWon']
211 1.3269429 ['matchLost']
229 2.3392708 ['uiInteraction']
359 1.7847396 ['matchStarted']
531 1.9186443 ['adShow']
532 1.7558193 ['adClosed']
536 1.1789985 ['outOfGameSend']
540 1.6750678 ['adRequest']
545 1

In [67]:
data[data['new_event_id']==11118]
data[data['new_game_id']==101].drop_duplicates('eventname')

,gameid,genre,platform,eventname,eventparameter,eventtype,new_game_id,new_event_id
82191,2976,No genre,pc,increaseXP,playerCoins,custom,101,1112
82192,2976,No genre,pc,clientDevice,deviceType,standard,101,9
82196,2976,No genre,pc,gameStarted,userLocale,standard,101,12
82202,2976,No genre,pc,chatReceiveMessage,userScore,custom,101,1113
82203,2976,No genre,pc,stageUnlocked,userScore,custom,101,1114
82204,2976,No genre,pc,testAborted,userScore,custom,101,1115
82205,2976,No genre,pc,block100Build,userScore,custom,101,1116
82206,2976,No genre,pc,challengeTimeout,userScore,custom,101,1117
82207,2976,No genre,pc,toolDashUse,userScore,custom,101,1118
82208,2976,No genre,pc,timeTrialNewRecord,userScore,custom,101,1119
